# Deep Learning -- Convolutional Neural Network (CNN) for Handwritten Digits Recognition

The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 

Follow instructions step by step until the end and submit your complete notebook as an archive (`tar -cf groupXnotebook.tar DL_lab2/`).

Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by the deadline on Moodle.

In the last lab session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST dataset. The best achieved accuracy on testing data was >90%.  Can  you do better than these results using a deep CNN?
Now, let's build a more interesting model. In this notebook, you will build, train and optimize in TensorFlow one of the simplest Convolutional Neural Networks, **LeNet-5**, proposed by Yann LeCun, Leon Bottou, Yosuha Bengio and Patrick Haffner in 1998 (for more details, check the paper *"Gradient-Based Learning Applied to Document Recognition"*, Y.LeCun et al.).


Since, it's possible that this is our first time using Tensorflow, the next section will serve as introduction to this framework (*Note*: we will use TF 1.x).
**Please, even if there is no coding required -- do NOT skip it and READ carefully everything.**

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Introduction to Tensorflow 101
## TensorFlow Static Graph

The entire purpose of Tensorflow is to have a so-called computational graph that can be executed much more efficiently than if the same calculations were to be performed directly in Python. TensorFlow can be more efficient than NumPy because TensorFlow knows the entire computation graph that must be executed, while NumPy only knows the computation of a single mathematical operation at a time.

TensorFlow can also automatically calculate the gradients that are needed to optimize the variables of the graph so as to make the model perform better. This is because the graph is a combination of simple mathematical expressions so the gradient of the entire graph can be calculated using the chain-rule for derivatives.

TensorFlow can also take advantage of multi-core CPUs as well as GPUs - and Google has even built special hardware accelerators just for TensorFlow which are called TPUs (Tensor Processing Units) that are even faster than GPUs.

A TensorFlow graph consists of the following parts which will be detailed below:

- Placeholder variables used to feed input into the graph.
- Model variables that are going to be optimized so as to make the model perform better.
- The model which is essentially just a mathematical function that calculates some output given the input in the placeholder variables and the model variables.
- A cost measure that can be used to guide the optimization of the variables.
- An optimization method which updates the variables of the model.

In addition, the TensorFlow graph may also contain various debugging statements e.g. for logging data to be displayed using TensorBoard.

## Placeholder variables
Placeholder variables serve as the input to the graph that we may change each time we execute the graph. We call this feeding the placeholder variables and it is demonstrated further below.

First we define the placeholder variable for the input images. This allows us to change the images that are input to the TensorFlow graph. This is a so-called tensor, which just means that it is a multi-dimensional vector or matrix. The data-type is set to float32 and the shape is set to `[None, img_size_flat]`, where None means that the tensor may hold an arbitrary number of images with each image being a vector of length `img_size_flat` (in our case it's 784).

In [2]:
x = tf.placeholder(tf.float32, [None, 784], name='inputs')
print(x)

Tensor("inputs:0", shape=(?, 784), dtype=float32)


Next we have the placeholder variable for the true labels associated with the images that were input in the placeholder variable x. 
The shape of this placeholder variable is `[None, num_classes]` which means it may hold an arbitrary number of labels and each label is a vector of length `num_classes` which is 10 in this case.

In [3]:
y_true = tf.placeholder(tf.float32, [None, 10], name='labels')
print(y_true)

Tensor("labels:0", shape=(?, 10), dtype=float32)


Finally we have the tensor variable for the true class of each image in the placeholder variable `x`. These are integers and the dimensionality of this placeholder variable is set to `[None]` which means the placeholder variable is a one-dimensional vector of arbitrary length.

In [4]:
y_true_cls = tf.argmax(y_true, 1) 

## Variables to be optimized

Apart from the placeholder variables that were defined above and which serve as feeding input data into the model, there are also some model variables that must be changed by TensorFlow so as to make the model perform better on the training data.

The first variable that must be optimized is called `weights` and is defined here as a TensorFlow variable that must be initialized with zeros and whose shape is `[img_size_flat, num_classes]`, so it is a 2-dimensional tensor (or matrix) with `img_size_flat` rows and `num_classes` columns.

In [5]:
weights = tf.Variable(tf.zeros([784, 10]), name='weights')
print(weights)

<tf.Variable 'weights:0' shape=(784, 10) dtype=float32_ref>


The second variable that must be optimized is called `biases` and is defined as a 1-dimensional tensor (or vector) of length `num_classes`.

In [6]:
biases = tf.Variable(tf.zeros([10]), name='bias')
print(biases)

<tf.Variable 'bias:0' shape=(10,) dtype=float32_ref>


## Model

This simple mathematical model multiplies the images in the placeholder variable `x` with the `weights` and then adds the `biases`.

The result is a matrix of shape `[num_images, num_classes]` because `x` has shape `[num_images, img_size_flat]` and `weights` has shape `[img_size_flat, num_classes]`, so the multiplication of those two matrices is a matrix with shape `[num_images, num_classes]` and then the `biases` vector is added to each row of that matrix.

Note that the name `logits` is typical TensorFlow terminology, but other people may call the variable something else.

Now `logits` is a matrix with `num_images` rows and `num_classes` columns, where the element of the $i$'th row and $j$'th column is an estimate of how likely the $i$'th input image is to be of the $j$'th class.

However, these estimates are a bit rough and difficult to interpret because the numbers may be very small or large, so we want to normalize them so that each row of the `logits` matrix sums to one, and each element is limited between zero and one. This is calculated using the so-called softmax function and the result is stored in `y_pred`.

The predicted class can be calculated from the `y_pred` matrix by taking the index of the largest element in each row.

In [7]:
with tf.name_scope('model'):
    logits = tf.matmul(x, weights) + biases
    y_pred = tf.nn.softmax(logits)
    y_pred_cls = tf.argmax(y_pred, axis=1)

## Cost-function to be optimized

To make the model better at classifying the input images, we must somehow change the variables for `weights` and `biases`. To do this we first need to know how well the model currently performs by comparing the predicted output of the model `y_pred` to the desired output `y_true`.

The cross-entropy is a performance measure used in classification. The cross-entropy is a continuous function that is always positive and if the predicted output of the model exactly matches the desired output then the cross-entropy equals zero. The goal of optimization is therefore to minimize the cross-entropy so it gets as close to zero as possible by changing the `weights` and `biases` of the model.

TensorFlow has a built-in function for calculating the cross-entropy. Note that it uses the values of the `logits` because it also calculates the softmax internally.

After that, we have the cross-entropy for each of the image classifications so we have a measure of how well the model performs on each image individually. But in order to use the cross-entropy to guide the optimization of the model's variables we need a single scalar value, so we simply take the average of the cross-entropy for all the image classifications.

In [8]:
with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                               labels=y_true)
    loss = tf.reduce_mean(cross_entropy)

## Optimization
Now that we have a cost measure that must be minimized, we can then create an optimizer. In this case it is the basic form of Gradient Descent where the step-size is set to 0.01.

Note that optimization is not performed at this point. In fact, nothing is calculated at all, we just add the optimizer-object to the TensorFlow graph for later execution.

In [9]:
learning_rate = 0.01
with tf.name_scope('optim'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    opt_step = optimizer.minimize(loss)

## Performance 

We need a few more performance measures to display the progress to the user.

This is a vector of booleans whether the predicted class equals the true class of each image.

This calculates the classification accuracy by first type-casting the vector of booleans to floats, so that False becomes 0 and True becomes 1, and then calculating the average of these numbers.

In [10]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


## TensorFlow Session

Once the TensorFlow graph has been created, we have to create a TensorFlow session which is used to execute the graph.

In [11]:
session = tf.Session()

### Initialize variables
The variables for `weights` and `biases` must be initialized before we start optimizing them.

In [12]:
init_op = tf.global_variables_initializer()
session.run(init_op)

### Setup the TensorBoard

Tensorboard is shipped with TensorFlow and it's a tool that allows to plot metrics, debug the graph, and much more.

In [13]:
def next_path(path_pattern):
    import os
    i = 1
    while os.path.exists(path_pattern % i):
        i = i * 2
    a, b = (i / 2, i)
    while a + 1 < b:
        c = (a + b) / 2 
        a, b = (c, b) if os.path.exists(path_pattern % c) else (a, c)
    directory = path_pattern % b
    return directory


writer = tf.summary.FileWriter(next_path('logs/run_%02d'))
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()


## Time to learn

Now that everything is defined, we can move to running the optimization.
There are 55.000 images in the training-set. It takes a long time to calculate the gradient of the model using all these images. We therefore use Stochastic Gradient Descent which only uses a small batch of images in each iteration of the optimizer.

In [14]:
batch_size = 100

Function for performing a number of optimization iterations so as to gradually improve the `weights` and `biases` of the model. In each iteration, a new batch of data is selected from the training-set and then TensorFlow executes the optimizer using those training samples. 

Let's define a couple of functions that will be usefull later.

In [15]:
def optimize(epochs):
    # Go through the traning dataset `epochs` times
    for e in range(epochs):
        
        num_of_batches = int(mnist.train.num_examples/batch_size)
        
        # We save also the loss across all the batches of data for 
        # presentation purpose
        avg_loss = 0.
        
        # Loop over all batches
        for i in range(num_of_batches):
            # Get a batch of training examples (shuffle every epoch).
            # x_batch now holds a batch of images and
            # y_true_batch are the true labels for those images.
            x_batch, y_true_batch = mnist.train.next_batch(batch_size, shuffle=(i==0))
            
            # Put the batch into a dict with the proper names
            # for placeholder variables in the TensorFlow graph.
            # Note that the placeholder for y_true_cls is not set
            # because it is not used during training.
            feed_dict_train = {x: x_batch,
                               y_true: y_true_batch}
            
            # Run the optimizer using this batch of training data.
            # TensorFlow assigns the variables in feed_dict_train
            # to the placeholder variables and then runs the optimizer.
            session.run(opt_step, feed_dict=feed_dict_train)
            
            # Similarly, get the loss and accuracy metrics on the batch of data
            batch_loss, summary = session.run([loss, merged_summary_op], feed_dict=feed_dict_train)
            
            # Write logs at every iteration
            writer.add_summary(summary, e * num_of_batches + i)
            
            # Compute average loss
            avg_loss += batch_loss / num_of_batches
            
        print("Epoch: ", '%02d' % (e + 1), "  =====> Loss =", "{:.6f}".format(avg_loss))

def print_accuracy():
    feed_dict_test = {x: mnist.test.images,
                  y_true: mnist.test.labels}
    
    # Use TensorFlow to compute the accuracy.
    # We are also going to save some metric like memory usage and computation time
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    acc = session.run(accuracy, feed_dict=feed_dict_test, options=run_options, run_metadata=run_metadata)
    try:
        writer.add_run_metadata(run_metadata, 'inference')
    except ValueError:
        pass
    writer.flush()
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

Let's add the graph to TensorBoard for easy debugging

In [16]:
writer.add_graph(tf.get_default_graph())

### Performance before any optimization

The accuracy on the test-set is 9.8%. This is because the model has only been initialized and not optimized at all, so it always predicts that the image shows a zero digit and it turns out that 9.8% of the images in the test-set happens to be zero digits.

In [17]:
print_accuracy()

Accuracy on test-set: 9.8%


Now we can train the model for 50 epochs and print the accuracy.

In [18]:
optimize(50)
print_accuracy()

Epoch:  01   =====> Loss = 1.176233
Epoch:  02   =====> Loss = 0.662504
Epoch:  03   =====> Loss = 0.550655
Epoch:  04   =====> Loss = 0.496804
Epoch:  05   =====> Loss = 0.463772
Epoch:  06   =====> Loss = 0.440938
Epoch:  07   =====> Loss = 0.423967
Epoch:  08   =====> Loss = 0.410685
Epoch:  09   =====> Loss = 0.399935
Epoch:  10   =====> Loss = 0.390951
Epoch:  11   =====> Loss = 0.383337
Epoch:  12   =====> Loss = 0.376758
Epoch:  13   =====> Loss = 0.371061
Epoch:  14   =====> Loss = 0.365974
Epoch:  15   =====> Loss = 0.361400
Epoch:  16   =====> Loss = 0.357319
Epoch:  17   =====> Loss = 0.353593
Epoch:  18   =====> Loss = 0.350193
Epoch:  19   =====> Loss = 0.347047
Epoch:  20   =====> Loss = 0.344152
Epoch:  21   =====> Loss = 0.341471
Epoch:  22   =====> Loss = 0.338952
Epoch:  23   =====> Loss = 0.336701
Epoch:  24   =====> Loss = 0.334475
Epoch:  25   =====> Loss = 0.332478
Epoch:  26   =====> Loss = 0.330563
Epoch:  27   =====> Loss = 0.328733
Epoch:  28   =====> Loss = 0

Using Tensorboard, we can now visualize the created graph, giving you an overview of your architecture and how all of the major components are connected. You can also see and analyse the learning curves. 

To launch TensorBoard: 
- Open a Terminal and run the command line `tensorboard --logdir logs`
- Click on "Tensorboard web interface" in Zoe (available only within EURECOM network) 

Try to play with it: check the behaviour of the training procedure, check the graph and try to match the nodes with their definition in the code, check the profiling that we did for the inference (how much memory it requires, how long it takes, ...).

<b> I unfortunatly can't use TensorBoard even with the instructions given... There would not be any graph, sorry. </b>

# LeNet5

You are now familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png" width="800" height="600" align="center">





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [19]:
# Functions for weigths and bias initilization 
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.constant(0., shape=shape)
    return tf.Variable(initial, name=name)

In [22]:
from tensorflow.contrib.layers import flatten

def build_lenet5(x):
    with tf.name_scope("reshape"):
        image = tf.reshape(x, [-1, 28, 28, 1]) # [None, 28, 28, 1]
    
    with tf.name_scope("lenet5"):
        with tf.name_scope("layer1"):
           ## --- COMPLETE --- ##
            conv1_W = weight_variable((5, 5, 1, 6), "conv1_W")
            conv1_b = bias_variable([1, 28, 28, 6], "bias1")
            conv1 = tf.nn.conv2d(input = image, filter = conv1_W, strides = [1, 1, 1, 1], padding = 'SAME') + conv1_b
            relu1 = tf.nn.relu(conv1)
            max_pool1 = tf.nn.max_pool(value = relu1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')


        with tf.name_scope("layer2"):
            ## --- COMPLETE --- ##
            conv2_W = weight_variable((5, 5, 6, 16), "conv2_W")
            conv2_b = bias_variable([1, 10, 10, 16], "bias2")
            conv2 = tf.nn.conv2d(input = max_pool1, filter = conv2_W, strides = [1, 1, 1, 1], padding = 'VALID') + conv2_b
            relu2 = tf.nn.relu(conv2)
            max_pool2 = tf.nn.max_pool(value = relu2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')
        
        
        with tf.name_scope("flatten"):
            ## --- COMPLETE --- ##
            flat = flatten(max_pool2)
            
            
        with tf.name_scope("layer3"):
            ## --- COMPLETE --- ##
            layer3_W = weight_variable((400, 120), "layer3_W")
            layer3_b = tf.Variable(tf.zeros([120]), name='bias3')
            fc3 = tf.matmul(flat, layer3_W) + layer3_b
            o_layer3 = tf.nn.relu(fc3)
            
            
        with tf.name_scope("layer4"):
            ## --- COMPLETE --- ##    
            layer4_W = weight_variable((120, 84), "layer4_W")
            layer4_b = tf.Variable(tf.zeros([84]), name='bias4')
            fc4 = tf.matmul(o_layer3, layer4_W) + layer4_b
            o_layer4 = tf.nn.relu(fc4)
            

        with tf.name_scope("layer5"):
            ## --- COMPLETE --- ##
            layer5_W = weight_variable((84, 10), "layer5_W")
            layer5_b = tf.Variable(tf.zeros([10]), name='bias5')            
            fc5 = tf.matmul(o_layer4, layer5_W) + layer5_b

    
    return fc5

<b> Question  </b>  Calculate the number of parameters of this model 

The number of parameters can be calculating thanks to the caracteristics of the layers we have (we calculate the parameters of the convolution layers, activations layer, pooling layers and Fully Connected layers), by additionning the number of wieghts and the number of biases: 
    * Input layer: 0 parameter
    * Layer 1: 5x5x1x6 + 6 = 156 parameters
    * Layer 2: 5x5x6x16 + 16 = 2 416 parameters
    * Layer 3: 400x120 + 120 =  48 120 parameters
    * Layer 4: 120x84 + 84 = 10 164 parameters
    * Layer 5: 84x10 + 10 = 850 parameters
In adiddition, the total number of parameters is <b> 61 706 parameters </b> . 

<b> Question </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [23]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
batch_size = 128

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='inputs')
y_true = tf.placeholder(tf.float32, [None, 10], name='labels')
y_true_cls = tf.argmax(y_true, 1) 

with tf.name_scope('model'):
    logits = build_lenet5(x)   ## --- COMPLETE --- ##
    y_pred = tf.nn.softmax(logits)   ## --- COMPLETE --- ##
    y_pred_cls = tf.argmax(y_pred, axis=1)   ## --- COMPLETE --- ##

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_true)   ## --- COMPLETE --- ##
    loss = tf.reduce_mean(cross_entropy)   ## --- COMPLETE --- ##
    
with tf.name_scope('optim'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)   ## --- COMPLETE --- ##
    #optimizer2 = tf.train.GradientDescentOptimizer(learning_rate)   ## --- COMPLETE --- ##
    opt_step = optimizer.minimize(loss)   ## --- COMPLETE --- ##
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)   ## --- COMPLETE --- ##
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))   ## --- COMPLETE --- ##

session = tf.Session()   ## --- COMPLETE --- ##
init_op = tf.global_variables_initializer()   ## --- COMPLETE --- ##
session.run(init_op)

writer = tf.summary.FileWriter(next_path('logs/lenet5/run_%02d'))
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

writer.add_graph(tf.get_default_graph())

**Question:**
Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [24]:
## --- COMPLETE --- ##

def optimize2(epochs):
    # Go through the traning dataset `epochs` times
    for e in range(epochs):
        
        num_of_batches = int(mnist.train.num_examples/batch_size)
        
        # We save also the loss across all the batches of data for 
        # presentation purpose
        avg_loss = 0.
        
        # Loop over all batches
        for i in range(num_of_batches):
            # Get a batch of training examples (shuffle every epoch).
            # x_batch now holds a batch of images and
            # y_true_batch are the true labels for those images.
            x_batch, y_true_batch = mnist.train.next_batch(batch_size, shuffle=(i==0))
            
            # Put the batch into a dict with the proper names
            # for placeholder variables in the TensorFlow graph.
            # Note that the placeholder for y_true_cls is not set
            # because it is not used during training.
            feed_dict_train = {x: x_batch,
                               y_true: y_true_batch}

            # Run the optimizer using this batch of training data.
            # TensorFlow assigns the variables in feed_dict_train
            # to the placeholder variables and then runs the optimizer.
            session.run(opt_step, feed_dict=feed_dict_train)
            
            # Similarly, get the loss and accuracy metrics on the batch of data
            batch_loss, summary, acc_train = session.run([loss, merged_summary_op, accuracy], feed_dict=feed_dict_train)
            
            # Write logs at every iteration
            writer.add_summary(summary, e * num_of_batches + i)
            
            # Compute average loss
            avg_loss += batch_loss / num_of_batches
            
            
            feed_dict_validation = {x: mnist.validation.images,
                  y_true: mnist.validation.labels}
            acc_validation = session.run(accuracy, feed_dict=feed_dict_validation)


        print("Epoch: ", '%02d' % (e + 1), "  =====> Loss =", "{:.6f}".format(avg_loss), 
             ", Training Accuracy = ", acc_train, 
              ", Validation Accuracy = ", acc_validation)

def print_accuracy2():
    feed_dict_test = {x: mnist.test.images,
                  y_true: mnist.test.labels}
    
    # Use TensorFlow to compute the accuracy.
    # We are also going to save some metric like memory usage and computation time
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    acc = session.run(accuracy, feed_dict=feed_dict_test, options=run_options, run_metadata=run_metadata)
    try:
        writer.add_run_metadata(run_metadata, 'inference')
    except ValueError:
        pass
    writer.flush()
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [28]:
import time

writer.add_graph(tf.get_default_graph())

start = time.time()
epochs = 40
optimize2(epochs)
print("Training time (Gradient Descent optimizer): ", time.time() - start, "s")

print_accuracy2()

Epoch:  01   =====> Loss = 2.301843 , Training Accuracy =  0.1484375 Validation Accuracy =  0.1628
Epoch:  02   =====> Loss = 2.295971 , Training Accuracy =  0.25 Validation Accuracy =  0.1978
Epoch:  03   =====> Loss = 2.289341 , Training Accuracy =  0.25 Validation Accuracy =  0.2248
Epoch:  04   =====> Loss = 2.280553 , Training Accuracy =  0.296875 Validation Accuracy =  0.2606
Epoch:  05   =====> Loss = 2.267248 , Training Accuracy =  0.296875 Validation Accuracy =  0.3206
Epoch:  07   =====> Loss = 2.193868 , Training Accuracy =  0.5078125 Validation Accuracy =  0.479
Epoch:  08   =====> Loss = 2.064315 , Training Accuracy =  0.6171875 Validation Accuracy =  0.679
Epoch:  09   =====> Loss = 1.687519 , Training Accuracy =  0.71875 Validation Accuracy =  0.7454
Epoch:  10   =====> Loss = 1.077035 , Training Accuracy =  0.8125 Validation Accuracy =  0.7976
Epoch:  11   =====> Loss = 0.708863 , Training Accuracy =  0.84375 Validation Accuracy =  0.8396
Epoch:  12   =====> Loss = 0.54

<b> Question </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Please put your loss and accuracy curves here.

## Improve the LeNET 5 Optimization

In [28]:
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
batch_size = 128

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='inputs')
y_true = tf.placeholder(tf.float32, [None, 10], name='labels')
y_true_cls = tf.argmax(y_true, 1) 

with tf.name_scope('model'):
    logits = build_lenet5(x)   ## --- COMPLETE --- ##
    y_pred = tf.nn.softmax(logits)   ## --- COMPLETE --- ##
    y_pred_cls = tf.argmax(y_pred, axis=1)   ## --- COMPLETE --- ##

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_true)   ## --- COMPLETE --- ##
    loss = tf.reduce_mean(cross_entropy)   ## --- COMPLETE --- ##
    
with tf.name_scope('optim'):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)   ## --- COMPLETE --- ##
    opt_step = optimizer.minimize(loss)   ## --- COMPLETE --- ##
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)   ## --- COMPLETE --- ##
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))   ## --- COMPLETE --- ##

session = tf.Session()   ## --- COMPLETE --- ##
init_op = tf.global_variables_initializer()   ## --- COMPLETE --- ##
session.run(init_op)

writer = tf.summary.FileWriter(next_path('logs/lenet5/run_%02d'))
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

writer.add_graph(tf.get_default_graph())

In [29]:
import time

writer.add_graph(tf.get_default_graph())

start = time.time()
epochs = 40
optimize2(epochs)
print("Training time (Adam optimizer): ", time.time() - start, "s")

print_accuracy2()

Epoch:  01   =====> Loss = 0.319106 , Training Accuracy =  0.984375 , Validation Accuracy =  0.9684
Epoch:  02   =====> Loss = 0.084518 , Training Accuracy =  1.0 , Validation Accuracy =  0.9776
Epoch:  03   =====> Loss = 0.057625 , Training Accuracy =  0.9765625 , Validation Accuracy =  0.984
Epoch:  04   =====> Loss = 0.042855 , Training Accuracy =  0.984375 , Validation Accuracy =  0.986
Epoch:  05   =====> Loss = 0.033237 , Training Accuracy =  1.0 , Validation Accuracy =  0.9862
Epoch:  06   =====> Loss = 0.026538 , Training Accuracy =  0.9921875 , Validation Accuracy =  0.986
Epoch:  07   =====> Loss = 0.021137 , Training Accuracy =  0.9921875 , Validation Accuracy =  0.986
Epoch:  08   =====> Loss = 0.016859 , Training Accuracy =  1.0 , Validation Accuracy =  0.9858
Epoch:  09   =====> Loss = 0.014822 , Training Accuracy =  1.0 , Validation Accuracy =  0.9862
Epoch:  10   =====> Loss = 0.012773 , Training Accuracy =  0.9921875 , Validation Accuracy =  0.9872
Epoch:  11   =====> 


<b> Question </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         95,1%      |        99,0%        |       
| Training Time        |        8568s       |        4595s        |  

- Which optimizer gives the best accuracy on test data?

The <b> Adam optimizer </b> gives us, by far, the best accuracy on test data in less time.


<b> Question </b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use `tf.nn.dropout` for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** On the testing data, we achieved <b> 1.0 accuracy </b>. However, the accuracy on the testing set is similar to the one obtained without dropout. It doesn't allow neither the algorithm to go faster. Thus, adding dropout is not essential. 

In [33]:
def build_lenet5_dropout(x):    
    ## --- COMPLETE --- ##
    with tf.name_scope("reshape"):
        image = tf.reshape(x, [-1, 28, 28, 1]) # [None, 28, 28, 1]
    
    with tf.name_scope("lenet5"):
        with tf.name_scope("layer1"):
           ## --- COMPLETE --- ##
            conv1_W = weight_variable((5, 5, 1, 6), "conv1_W")
            conv1_b = bias_variable([1, 28, 28, 6], "bias1")
            conv1 = tf.nn.conv2d(input = image, filter = conv1_W, strides = [1, 1, 1, 1], padding = 'SAME') + conv1_b
            relu1 = tf.nn.relu(conv1)
            max_pool1 = tf.nn.max_pool(value = relu1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')


        with tf.name_scope("layer2"):
            ## --- COMPLETE --- ##
            conv2_W = weight_variable((5, 5, 6, 16), "conv2_W")
            conv2_b = bias_variable([1, 10, 10, 16], "bias2")
            conv2 = tf.nn.conv2d(input = max_pool1, filter = conv2_W, strides = [1, 1, 1, 1], padding = 'VALID') + conv2_b
            relu2 = tf.nn.relu(conv2)
            max_pool2 = tf.nn.max_pool(value = relu2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')
        
        
        with tf.name_scope("flatten"):
            ## --- COMPLETE --- ##
            flat = flatten(max_pool2)
            droupout = tf.nn.dropout(flat, keep_prob = 0.75)
            
            
        with tf.name_scope("layer3"):
            ## --- COMPLETE --- ##
            layer3_W = weight_variable((400, 120), "layer3_W")
            layer3_b = tf.Variable(tf.zeros([120]), name='bias3')
            fc3 = tf.matmul(droupout, layer3_W) + layer3_b
            o_layer3 = tf.nn.relu(fc3)
            
            
        with tf.name_scope("layer4"):
            ## --- COMPLETE --- ##    
            layer4_W = weight_variable((120, 84), "layer4_W")
            layer4_b = tf.Variable(tf.zeros([84]), name='bias4')
            fc4 = tf.matmul(o_layer3, layer4_W) + layer4_b
            o_layer4 = tf.nn.relu(fc4)
            

        with tf.name_scope("layer5"):
            ## --- COMPLETE --- ##
            layer5_W = weight_variable((84, 10), "layer5_W")
            layer5_b = tf.Variable(tf.zeros([10]), name='bias5')            
            fc5 = tf.matmul(o_layer4, layer5_W) + layer5_b

    
    return fc5







tf.reset_default_graph()
## --- COMPLETE --- ##
tf.reset_default_graph() # reset the default graph before defining a new model

# Parameters
learning_rate = 0.001
batch_size = 128

# Model, loss function and accuracy
x = tf.placeholder(tf.float32, [None, 784], name='inputs')
y_true = tf.placeholder(tf.float32, [None, 10], name='labels')
y_true_cls = tf.argmax(y_true, 1) 

with tf.name_scope('model'):
    logits = build_lenet5_dropout(x)   ## --- COMPLETE --- ##
    y_pred = tf.nn.softmax(logits)   ## --- COMPLETE --- ##
    y_pred_cls = tf.argmax(y_pred, axis=1)   ## --- COMPLETE --- ##

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_true)   ## --- COMPLETE --- ##
    loss = tf.reduce_mean(cross_entropy)   ## --- COMPLETE --- ##
    
with tf.name_scope('optim'):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)   ## --- COMPLETE --- ##
    opt_step = optimizer.minimize(loss)   ## --- COMPLETE --- ##
    
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)   ## --- COMPLETE --- ##
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))   ## --- COMPLETE --- ##

session = tf.Session()   ## --- COMPLETE --- ##
init_op = tf.global_variables_initializer()   ## --- COMPLETE --- ##
session.run(init_op)

writer = tf.summary.FileWriter(next_path('logs/lenet5/run_%02d'))
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

writer.add_graph(tf.get_default_graph())


start = time.time()
epochs = 40
optimize2(epochs)
print("Training time (Adam optimizer): ", time.time() - start, "s")

print_accuracy2()

Epoch:  01   =====> Loss = 0.406412 , Training Accuracy =  0.953125 , Validation Accuracy =  0.961
Epoch:  02   =====> Loss = 0.111905 , Training Accuracy =  0.9765625 , Validation Accuracy =  0.9716
Epoch:  03   =====> Loss = 0.076919 , Training Accuracy =  0.9765625 , Validation Accuracy =  0.9718
Epoch:  04   =====> Loss = 0.060198 , Training Accuracy =  0.9921875 , Validation Accuracy =  0.978
Epoch:  05   =====> Loss = 0.053101 , Training Accuracy =  0.96875 , Validation Accuracy =  0.9822
Epoch:  06   =====> Loss = 0.044314 , Training Accuracy =  0.9921875 , Validation Accuracy =  0.9814
Epoch:  07   =====> Loss = 0.038939 , Training Accuracy =  0.9921875 , Validation Accuracy =  0.9828
Epoch:  08   =====> Loss = 0.035548 , Training Accuracy =  0.9765625 , Validation Accuracy =  0.9864
Epoch:  09   =====> Loss = 0.031038 , Training Accuracy =  0.984375 , Validation Accuracy =  0.987
Epoch:  10   =====> Loss = 0.028235 , Training Accuracy =  0.984375 , Validation Accuracy =  0.985